In [ ]:

import yaml
import pandas as pd
import requests
import io
import os
import matplotlib.pyplot as plt
import seaborn as sns

# create folders to save data (git does not track empty directories)

os.makedirs('data/', exist_ok=True)
os.makedirs('data/processed/', exist_ok=True)  
os.makedirs('data/raw/', exist_ok=True)  

# load login credentials

with open("config/login.yaml", "r", encoding="utf-8") as file:
    conf = yaml.safe_load(file)

config = {
    "path": conf["access_path"],
    "format": conf["format"]
}

login = {
    "username": conf["username"],
    "password": conf["password"]
}


## Microcensus data

The microcensus is a survey conducted by the national statistical office that collects data about a fraction of the total national population. Let us look at which data tables exist for the microcensus:

In [ ]:
def get_table_list(selection, parameters, config):
    parameters["selection"] = selection
    url = f"{config['path']}/catalogue/tables"
    return requests.get(url, params=parameters).json()

In [ ]:
relevant_tables = get_table_list("12211*", login, config)
relevant_tables

Let us look at the data for a particular table, e.g. `12211-0003` looks potentially interesting because it has info on income. 

In [ ]:

def get_data(table, parameters, config):
    csv_path = f"data/raw/tablefile_{table}.csv"
    if os.path.isfile(csv_path):
            data = pd.read_csv(csv_path)
    else:
            data = get_data_from_api(table, parameters, config)
    return data

        
def get_data_from_api(table, parameters, config):
    print(f"Requesting table data for table '{table}'")
    parameters["name"] = table

    # use "ffcsv" format because it is consistent across tables
    # and over time
    parameters["format"] = config["format"]
    url = f"{config['path']}/data/tablefile"
    
    response = requests.get(url, params=parameters)
    if response.ok:
        data = pd.read_csv(
            io.StringIO(response.content.decode("utf-8")),
            sep = ";",
            decimal = ",",
            na_values = ["-"],
        )
        data.to_csv(f"data/raw/tablefile_{table}.csv", index=False)
        return data
    else:
        return response


In [ ]:
my_table = "12211-0003"

df = get_data(my_table, login, config)


In [ ]:
df.head()

## Data cleaning

In [ ]:
df.head()

In [ ]:
# filter out irrelevant rows, e.g., rows with 'Ohne Angabe' or 'Insgesamt'
df['BEV036__Bevoelkerung_in_Hauptwohnsitzhaushalten__1000'] = pd.to_numeric(df['BEV036__Bevoelkerung_in_Hauptwohnsitzhaushalten__1000'], errors='coerce')

df_filtered = df[df['2_Auspraegung_Label'].isin(['männlich', 'weiblich']) & ~df['3_Auspraegung_Label'].isin(['Ohne Angabe', 'Insgesamt'])]

# Group data by income bracket and gender, and sum up the relevant population or employment figures
grouped_data = df_filtered.groupby(['3_Auspraegung_Label', '2_Auspraegung_Label']).sum()

grouped_data.head()

In [ ]:
grouped_data.index

In [ ]:

# Extract unique income brackets
income_brackets = df_filtered['3_Auspraegung_Label'].unique()

# Plotting for each gender
genders = ['männlich', 'weiblich']
colors = ['blue', 'red']  # Colors for männlich and weiblich respectively

for gender in genders:
    plt.figure(figsize=(12, 6))
    population_values = []
    #total_population = grouped_data.loc[gender, 'BEV036__Bevoelkerung_in_Hauptwohnsitzhaushalten__1000'].sum()

    for bracket in income_brackets:
        if (bracket, gender) in grouped_data.index:
            population = grouped_data.loc[(bracket, gender), 'ERW041__Erwerbstaetige_aus_Hauptwohnsitzhaushalten__1000']
        else:
            population = 0  # Or use NaN if you prefer
        population_values.append(population)


In [ ]:
# Calculate the total population for each gender
total_population_by_gender = grouped_data.groupby(level=0).sum()

# Calculate percentage of total for each row
grouped_data['Percentage'] = grouped_data['BEV036__Bevoelkerung_in_Hauptwohnsitzhaushalten__1000'] / grouped_data.groupby(level=0).transform(sum)['BEV036__Bevoelkerung_in_Hauptwohnsitzhaushalten__1000'] * 100


In [ ]:
# Histogram using matplotlib

for gender in genders:
    plt.bar(income_brackets, population_values, color=colors[genders.index(gender)])
    plt.title(f'Employees in Main Residence Households for {gender} by Income Bracket')
    plt.xlabel('Income Bracket')
    plt.ylabel('Population in Thousands')
    plt.xticks(rotation=45)
    plt.show()

Can we see more if we look at relative numbers rather than sums of people?

In [ ]:

# Reset index for plotting
grouped_data_reset = grouped_data.reset_index()

# Plotting with Seaborn
plt.figure(figsize=(12, 6))
sns.barplot(data=grouped_data_reset, x='3_Auspraegung_Label', y='Percentage', hue='2_Auspraegung_Label', palette=['blue', 'red'])
plt.title('Percentage of Population in Main Residence Households by Income Bracket and Gender')
plt.xlabel('Income Bracket')
plt.ylabel('Percentage of Total Population')
plt.xticks(rotation=45)
plt.show()